In [ ]:
import numpy as np

In [25]:
def calculate_array_statistics(url, chunk_size=None):
    """
    Calculate running mean and standard deviation for a large memory-mapped array
    using chunked processing to manage memory usage.
    
    Parameters:
    -----------
    url : str
        Path to the .npy file
    chunk_size : int, optional
        Number of temporal samples to process at once. If None, will use 1000 * temporal_dimension
        
    Returns:
    --------
    tuple
        (running_mean, running_std) arrays of shape (channels,)
    """
    # Open memory-mapped array
    npy_array = np.lib.format.open_memmap(url, mode="r")
    total_samples, temporal, channels, height, width = npy_array.shape
    
    # Calculate effective batch size
    merged_batch_size = total_samples * temporal
    
    # Set default chunk size if not provided
    if chunk_size is None:
        chunk_size = 1000 * temporal
    
    # Initialize statistics trackers with explicit dtype
    running_mean = np.zeros(channels, dtype=np.float64)
    running_var = np.zeros(channels, dtype=np.float64)
    total_elements = 0
    
    # Process data in chunks
    for start in range(0, merged_batch_size, chunk_size):
        end = min(start + chunk_size, merged_batch_size)
        
        # Convert indices to sample and temporal ranges
        sample_start = start // temporal
        sample_end = (end - 1) // temporal + 1
        temporal_offset_start = start % temporal
        temporal_offset_end = end % temporal if end % temporal != 0 else temporal
        
        # Load and slice chunk
        chunk = npy_array[sample_start:sample_end]
        if sample_start == sample_end - 1:
            chunk = chunk[:, temporal_offset_start:temporal_offset_end]
        
        # Reshape for efficient computation
        reshaped = chunk.reshape(-1, channels, height, width).transpose(1, 0, 2, 3).reshape(channels, -1)
        
        # Update statistics using Welford's online algorithm with explicit type conversion
        chunk_mean = np.asarray(reshaped.mean(axis=1), dtype=np.float64)
        chunk_var = np.asarray(reshaped.var(axis=1), dtype=np.float64)
        chunk_elements = float(reshaped.shape[1])  # Convert to float for division
        
        
        total_elements += chunk_elements
        delta = chunk_mean - running_mean
        running_mean += delta * (chunk_elements / total_elements)
        running_var += (chunk_var * (chunk_elements / total_elements) + 
                        (delta**2) * (chunk_elements * (total_elements - chunk_elements)) / 
                        total_elements**2)
    
    running_std = np.sqrt(running_var)
    return running_mean, running_std

In [19]:
def print_statistics(url, chunk_size=None):
    """
    Calculate and print channel-wise statistics with proper formatting.
    """
    means, stds = calculate_array_statistics(url, chunk_size)
    
    print("\nChannel Statistics:")
    print("-" * 50)
    print(f"{'Channel':<10} {'Mean':>15} {'Std Dev':>15}")
    print("-" * 50)
    for i, (mean, std) in enumerate(zip(means, stds)):
        print(f"{i:<10} {mean:>15.6f} {std:>15.6f}")
    print("-" * 50)

In [ ]:
url = r"C:\Users\Andrew Deur\Documents\NYU\DS-GA 1008 Deep Learning\1008-Final-Proj\data\states.npy"
print_statistics(url, chunk_size=1000)


Channel Statistics:
--------------------------------------------------
Channel               Mean         Std Dev
--------------------------------------------------
0                 0.000237        0.003330
1                 0.009316        0.028159
--------------------------------------------------


In [26]:
print_statistics(url, chunk_size=10000)


Channel Statistics:
--------------------------------------------------
Channel               Mean         Std Dev
--------------------------------------------------
0                 0.000237        0.008225
1                 0.009316        0.069441
--------------------------------------------------
